In [ ]:
!pip install wandb

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import os

from tensorflow.keras.preprocessing import timeseries_dataset_from_array
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import datetime as dt

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow.keras.layers import LSTM, GRU, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, TimeDistributed, \
    BatchNormalization
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

import wandb
from wandb.keras import WandbCallback


In [3]:
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
tf.random.set_seed(hash("by removing stochasticity") % 2**32 - 1)

In [5]:
wandb.login()

wandb: Currently logged in as: nbvanting (use `wandb login --relogin` to force relogin)


True

In [6]:
def create_datasets(dataframe, split, steps, lookback, horizon, batch_size, scaler='standard'):
    # Split method
    train_split = int(split * dataframe.shape[0])
    val_split = int((split+0.1) * dataframe.shape[0])

    train = dataframe.iloc[:train_split]
    val = dataframe.iloc[train_split:val_split]
    test = dataframe.iloc[val_split:]

    # Scaler
    if scaler == 'standard':
        X_scaler = StandardScaler()
        y_scaler = StandardScaler()
    elif scaler == 'minmax':
        X_scaler = MinMaxScaler()
        y_scaler = MinMaxScaler()
    else:
        print("Please specify one of 'standard' or 'minmax' to scaler parameter.")
    
    # Training
    start = lookback + horizon
    end = start + train_split

    X_train = train.values
    y_train = dataframe.iloc[start:end][['Value']]

    X_train = X_scaler.fit_transform(X_train)
    y_train = y_scaler.fit_transform(y_train)

    # Validation
    x_end = len(val) - lookback - horizon
    y_val_start = train_split + lookback + horizon

    X_val = val.iloc[:x_end]
    y_val = dataframe.iloc[y_val_start:y_val_start+x_end][['Value']]

    X_val = X_scaler.transform(X_val)
    y_val = y_scaler.transform(y_val)

    # Test
    x_end = len(test) - lookback - horizon
    y_test_start = val_split + lookback + horizon

    X_test = test.iloc[:x_end]
    y_test = dataframe.iloc[y_test_start:y_test_start+x_end][['Value']]

    X_test = X_scaler.transform(X_test)
    y_test = y_scaler.transform(y_test)

    # Batch Sequence Generators
    sequence_length = int(lookback/steps)

    dataset_train = timeseries_dataset_from_array(
        X_train, y_train,
        sequence_length=sequence_length,
        sampling_rate=steps,
        batch_size=batch_size,
        shuffle=True
    )

    dataset_val = timeseries_dataset_from_array(
        X_val, y_val,
        sequence_length=sequence_length,
        sampling_rate=steps,
        batch_size=batch_size,
        shuffle=True
    )    

    dataset_test = timeseries_dataset_from_array(
        X_test, y_test,
        sequence_length=sequence_length,
        sampling_rate=steps,
        batch_size=batch_size,
        shuffle=True
    )    

    return dataset_train, dataset_val, dataset_test

def ts_offset_split(dataframe, steps, lookback, horizon, batch_size, scaler='standard'):
    start = 0
    end = 168
    offset = 24
    training = []
    validation = []

    for i in range(int((365+366)/8)):

        train = dataframe.iloc[start:end]
        val = dataframe.iloc[end:end+offset]
        training.append(train)
        validation.append(val)

        start += 192
        end += 192
    
    # Decide Splits for sets
    train = pd.concat(training)

    val = pd.concat(validation)
    train = train.append(dataframe[(dataframe.index.date > val.index.max()) & (dataframe.index.date < dt.date(2021,1,1))])

    test = dataframe[dataframe.index.date >= dt.date(2021,1,1)]
    
    tmpdf = pd.concat([train,val,test])

    # Scaler
    if scaler == 'standard':
        X_scaler = StandardScaler()
        y_scaler = StandardScaler()
    elif scaler == 'minmax':
        X_scaler = MinMaxScaler()
        y_scaler = MinMaxScaler()
    else:
        print("Please specify one of 'standard' or 'minmax' to scaler parameter.")
    
    # Training
    start = lookback + horizon
    end = start + train.shape[0]

    X_train = train.values
    y_train = tmpdf.iloc[start:end][['Value']]

    X_train = X_scaler.fit_transform(X_train)
    y_train = y_scaler.fit_transform(y_train)

    # Validation
    x_end = len(val) - lookback - horizon
    y_val_start = train.shape[0] + lookback + horizon

    X_val = val.iloc[:x_end]
    y_val = tmpdf.iloc[y_val_start:y_val_start+x_end][['Value']]

    X_val = X_scaler.transform(X_val)
    y_val = y_scaler.transform(y_val)

    # Test
    x_end = len(test) - lookback - horizon
    y_test_start = (train.shape[0] + val.shape[0]) + lookback + horizon

    X_test = test.iloc[:x_end]
    y_test = tmpdf.iloc[y_test_start:y_test_start+x_end][['Value']]

    X_test = X_scaler.transform(X_test)
    y_test = y_scaler.transform(y_test)

    # Batch Sequence Generators
    sequence_length = int(lookback/steps)

    dataset_train = timeseries_dataset_from_array(
        X_train, y_train,
        sequence_length=sequence_length,
        sampling_rate=steps,
        batch_size=batch_size,
        shuffle=True
    )

    dataset_val = timeseries_dataset_from_array(
        X_val, y_val,
        sequence_length=sequence_length,
        sampling_rate=steps,
        batch_size=batch_size,
        shuffle=True
    )    

    dataset_test = timeseries_dataset_from_array(
        X_test, y_test,
        sequence_length=sequence_length,
        sampling_rate=steps,
        batch_size=batch_size,
        shuffle=False
    )    

    return dataset_train, dataset_val, dataset_test

In [14]:
def load_data(config, splitmethod):
    # Load csv & parse dates to datetime index
    data = pd.read_csv(f'/content/drive/MyDrive/Thesis/{config.dataset}_features.csv', index_col='Datetime', parse_dates=['Datetime'])
    # Select Features
    data = data[['Value', 'sunshine_mins', 'airtemp_c', 'daylength_hrs', 'wkdy_sin', 'wkdy_cos', 'wknd', 'mnth_sin', 'mnth_cos']]
    
    if splitmethod == 'offset':
        train, val, test = ts_offset_split(data, 
                                  steps=steps, lookback=config.lookback, 
                                  horizon=horizon, batch_size=config.batch_size, 
                                  scaler='standard')
    
    elif splitmethod == 'standard':
        train, val, test = create_datasets(data, split=config.splitrate, 
                                  steps=steps, lookback=config.lookback, 
                                  horizon=horizon, batch_size=config.batch_size, 
                                  scaler='standard')
    
    return train, val, test
    
    
def build_model(config):
        
    model = Sequential()

    model.add(Input(shape=(config.lookback, config.num_features)))

    # CNN Block
    model.add(Conv1D(filters=config.cnn_layer_size_1, kernel_size=config.kernelsize, activation=config.activation_cnn))
    model.add(MaxPooling1D(pool_size=2))    
    if config.cnn_lay_2:
        model.add(Conv1D(filters=config.cnn_layer_size_2, kernel_size=config.kernelsize, activation=config.activation_cnn))
        model.add(MaxPooling1D(pool_size=2))
    if config.cnn_lay_3:
        model.add(Conv1D(filters=config.cnn_layer_size_3, kernel_size=config.kernelsize, activation=config.activation_cnn))
        model.add(MaxPooling1D(pool_size=2))
    
    # RNN Block
    if config.gru_lay_1:
        model.add(GRU(config.gru_layer_size_1, return_sequences=True, activation=config.activation_gru))
        model.add(Dropout(config.dropout))
    if config.gru_lay_2:
        model.add(GRU(config.gru_layer_size_2, return_sequences=True, activation=config.activation_gru))
        model.add(Dropout(config.dropout))
    
    model.add(GRU(config.gru_layer_size_3, return_sequences=False, activation=config.activation_gru))
    model.add(Dropout(config.dropout))

    model.add(Dense(1))
    
    opt = config.optimizer
    if opt == 'sgd':
        optimizer = SGD(learning_rate=config.learning_rate, momentum=config.momentum)
        
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    
    return model

def log_defaults():
    # Default values from the first sweep
    # The following values resulted in the strongest model
    wandb_config = {
        'num_features' : 9,
        'epochs' : 100,
        'batch_size' : 240,
        'num_cnn_layers' : 2, # Number of additional layers
        'num_gru_layers' : 2, # Number of additional layers
        'optimizer' : 'sgd',
        'dropout' : 0.024,
        'lookback' : 744,
        'activation_cnn' : 'relu',
        'activation_gru' : 'tanh',
        'kernelsize': 3,
        'cnn_lay_2' : True,
        'cnn_lay_3' : True,
        'cnn_layer_size_1' : 64,
        'cnn_layer_size_2' : 176,
        'cnn_layer_size_3' : 248,
        'gru_lay_1': True,
        'gru_lay_2': True,
        'gru_layer_size_1' : 56,
        'gru_layer_size_2' : 128,
        'gru_layer_size_3' : 144,
        'learning_rate' : 0.03515,
        'momentum' : 0.9,
        'dataset' : 'kolding',
        'splitrate': 0.8,
        'splitmethod': 'offset',
        
    }
    return wandb_config

def run_tuner():
    
    wandb.init(config=log_defaults(), group='cnnrnn-sweep-5', project='thesis')
    
    model = build_model(config=wandb.config)
    
    train, val, test = load_data(config=wandb.config, splitmethod=wandb.config.splitmethod)
    
    callbacks = [WandbCallback()]
    
    model.fit(
        train,
        epochs=wandb.config.epochs,
        validation_data=val,
        callbacks=callbacks
    )

    eval = model.evaluate(test)

    wandb.log({'test_loss': eval[0]})
    wandb.log({'test_mae': eval[1]})
    wandb.log({'test_mape': eval[2]})


In [10]:
# Sweep Config
sweep_config = {
    'method': 'random',
    'metric': {
        'name': 'val_loss',
        'goal': 'minimize'
    },
    'early_terminate': {
        'type': 'hyperband',
        'min_iter': 5
    },
    'parameters': {
        'num_features' : {
            'value' : 9
        },
        'batch_size': {
            'distribution': 'q_log_uniform',
            'q': 1,
            'min': math.log(32),
            'max': math.log(256)
        },
        'lookback': {
            'value': 24*31
        },
        'optimizer': {
            'value': 'sgd'
        },
        'dropout': {
            'distribution': 'uniform',
            'min': 0.01,
            'max': 0.5
        },
        'epochs': {
            'value': 25
        },
        'activation_gru': {
            'value': 'tanh'
        },
        'activation_cnn': {
            'value': 'relu'
        },
        'kernelsize': {
            'values': [3, 5, 7]
        },
        'cnn_lay_2': {
            'values': [True, False]
        },
        'cnn_lay_3': {
            'values': [True, False]
        },
        'cnn_layer_size_1': {
            'distribution': 'q_log_uniform',
            'q': 1,
            'min': math.log(32),
            'max': math.log(256)
        },
        'cnn_layer_size_2': {
            'distribution': 'q_log_uniform',
            'q': 1,
            'min': math.log(32),
            'max': math.log(256)
        },
        'cnn_layer_size_3': {
            'distribution': 'q_log_uniform',
            'q': 1,
            'min': math.log(32),
            'max': math.log(256)
        },
        'gru_lay_1': {
            'values': [True, False]
        },
        'gru_lay_2': {
            'values': [True, False]
        },
        'gru_layer_size_1': {
            'distribution': 'q_log_uniform',
            'q': 1,
            'min': math.log(32),
            'max': math.log(256)
        },
        'gru_layer_size_2': {
            'distribution': 'q_log_uniform',
            'q': 1,
            'min': math.log(32),
            'max': math.log(256)
        },
        'gru_layer_size_3': {
            'distribution': 'q_log_uniform',
            'q': 1,
            'min': math.log(32),
            'max': math.log(256)
        },
        'learning_rate': {
            'distribution': 'uniform',
            'min': 0.001,
            'max': 0.04
        },
        'momentum': {
            'distribution': 'uniform',
            'min': 0.5,
            'max': 1
        },
        'num_cnn_layers': {
            'value': 2
        },
        'num_gru_layers': {
            'value': 2
        },
        'dataset': {
            'values': ['kolding', 'strib']
        },
        'splitrate': {
            'value': 0.8
        },
        'splitmethod': {
            'values': ['offset', 'standard']
        }
    }
}

In [11]:
# Additional Parameters
steps = 1 # timesteps: 1 hour
horizon = 1 # the target hour in the future we want to predict 1 hour ahead



In [12]:
sweep_id = wandb.sweep(sweep_config, project='thesis')

Create sweep with ID: xjdtj3eo
Sweep URL: https://wandb.ai/nbvanting/thesis/sweeps/xjdtj3eo


In [15]:
wandb.agent(sweep_id, function=run_tuner, count=50)

wandb: Agent Starting Run: y60zb78r with config:
wandb: 	activation_cnn: relu
wandb: 	activation_gru: tanh
wandb: 	batch_size: 225
wandb: 	cnn_lay_2: True
wandb: 	cnn_lay_3: True
wandb: 	cnn_layer_size_1: 98
wandb: 	cnn_layer_size_2: 68
wandb: 	cnn_layer_size_3: 130
wandb: 	dataset: kolding
wandb: 	dropout: 0.22815591667389837
wandb: 	epochs: 25
wandb: 	gru_lay_1: True
wandb: 	gru_lay_2: True
wandb: 	gru_layer_size_1: 228
wandb: 	gru_layer_size_2: 128
wandb: 	gru_layer_size_3: 106
wandb: 	kernelsize: 5
wandb: 	learning_rate: 0.0229111795481337
wandb: 	lookback: 744
wandb: 	momentum: 0.5120062271948083
wandb: 	num_cnn_layers: 2
wandb: 	num_features: 9
wandb: 	num_gru_layers: 2
wandb: 	optimizer: sgd
wandb: 	splitmethod: standard
wandb: 	splitrate: 0.8
wandb: WARNING Ignored wandb.init() arg project when running a sweep


Epoch 1/25
70/70 [==============================] - 46s 132ms/step - loss: 0.7608 - mae: 0.6833 - val_loss: 1.5652 - val_mae: 1.0037
Epoch 2/25
70/70 [==============================] - 8s 110ms/step - loss: 0.7002 - mae: 0.6608 - val_loss: 1.4619 - val_mae: 0.9789
Epoch 3/25
70/70 [==============================] - 8s 109ms/step - loss: 0.6799 - mae: 0.6514 - val_loss: 1.3441 - val_mae: 0.9466
Epoch 4/25
70/70 [==============================] - 8s 109ms/step - loss: 0.6453 - mae: 0.6373 - val_loss: 1.1634 - val_mae: 0.8834
Epoch 5/25
70/70 [==============================] - 8s 112ms/step - loss: 0.5658 - mae: 0.6036 - val_loss: 1.3174 - val_mae: 0.9039
Epoch 6/25
70/70 [==============================] - 8s 110ms/step - loss: 0.4015 - mae: 0.4952 - val_loss: 0.3929 - val_mae: 0.4998
Epoch 7/25
70/70 [==============================] - 8s 111ms/step - loss: 0.2817 - mae: 0.4103 - val_loss: 0.3924 - val_mae: 0.4600
Epoch 8/25
70/70 [==============================] - 8s 112ms/step - loss: 0

epoch,24
loss,0.0972
mae,0.2393
val_loss,0.28715
val_mae,0.37963
_runtime,244
_timestamp,1620378361
_step,26
best_val_loss,0.24964
best_epoch,17
test_loss,0.15452


epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
loss,██▇▇▆▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mae,███▇▇▅▄▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▇▇▆▇▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_mae,██▇▇▇▂▂▂▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁
_runtime,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇████
_timestamp,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇████
_step,▁▁▂▂▂▂▃▃▃▃▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇██
test_loss,▁
test_mae,▁


Run y60zb78r errored: IndexError('list index out of range')
wandb: ERROR Run y60zb78r errored: IndexError('list index out of range')
wandb: Agent Starting Run: fab2b46v with config:
wandb: 	activation_cnn: relu
wandb: 	activation_gru: tanh
wandb: 	batch_size: 45
wandb: 	cnn_lay_2: True
wandb: 	cnn_lay_3: True
wandb: 	cnn_layer_size_1: 56
wandb: 	cnn_layer_size_2: 47
wandb: 	cnn_layer_size_3: 76
wandb: 	dataset: kolding
wandb: 	dropout: 0.49639390516274945
wandb: 	epochs: 25
wandb: 	gru_lay_1: False
wandb: 	gru_lay_2: True
wandb: 	gru_layer_size_1: 189
wandb: 	gru_layer_size_2: 134
wandb: 	gru_layer_size_3: 35
wandb: 	kernelsize: 5
wandb: 	learning_rate: 0.015546077663184037
wandb: 	lookback: 744
wandb: 	momentum: 0.8929410313661245
wandb: 	num_cnn_layers: 2
wandb: 	num_features: 9
wandb: 	num_gru_layers: 2
wandb: 	optimizer: sgd
wandb: 	splitmethod: standard
wandb: 	splitrate: 0.8
wandb: WARNING Ignored wandb.init() arg project when running a sweep


...

Epoch 1/25
287/287 [==============================] - 13s 34ms/step - loss: 0.7176 - mae: 0.6655 - val_loss: 0.4380 - val_mae: 0.5098
Epoch 2/25
287/287 [==============================] - 9s 31ms/step - loss: 0.2979 - mae: 0.4242 - val_loss: 0.1965 - val_mae: 0.3444
Epoch 3/25
287/287 [==============================] - 9s 31ms/step - loss: 0.1698 - mae: 0.3192 - val_loss: 0.1635 - val_mae: 0.3179
Epoch 4/25
287/287 [==============================] - 9s 30ms/step - loss: 0.1440 - mae: 0.2936 - val_loss: 0.1785 - val_mae: 0.3340
Epoch 5/25
287/287 [==============================] - 9s 30ms/step - loss: 0.1284 - mae: 0.2761 - val_loss: 0.1739 - val_mae: 0.3283
Epoch 6/25
287/287 [==============================] - 9s 30ms/step - loss: 0.1188 - mae: 0.2659 - val_loss: 0.1471 - val_mae: 0.3017
Epoch 7/25
287/287 [==============================] - 9s 30ms/step - loss: 0.1113 - mae: 0.2577 - val_loss: 0.1491 - val_mae: 0.3042
Epoch 8/25
287/287 [==============================] - 9s 30ms/step -

epoch,24
loss,0.07556
mae,0.21125
val_loss,0.12964
val_mae,0.28257
_runtime,237
_timestamp,1620388573
_step,26
best_val_loss,0.12667
best_epoch,22
test_loss,0.22223


epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
loss,█▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mae,█▄▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▃▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_mae,█▃▂▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_runtime,▁▁▂▂▂▂▃▃▃▄▄▄▄▅▅▅▅▆▆▆▇▇▇████
_timestamp,▁▁▂▂▂▂▃▃▃▄▄▄▄▅▅▅▅▆▆▆▇▇▇████
_step,▁▁▂▂▂▂▃▃▃▃▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇██
test_loss,▁
test_mae,▁


Run y9a0z7pz errored: IndexError('list index out of range')
wandb: ERROR Run y9a0z7pz errored: IndexError('list index out of range')
wandb: Agent Starting Run: 7wnco6y8 with config:
wandb: 	activation_cnn: relu
wandb: 	activation_gru: tanh
wandb: 	batch_size: 209
wandb: 	cnn_lay_2: False
wandb: 	cnn_lay_3: False
wandb: 	cnn_layer_size_1: 88
wandb: 	cnn_layer_size_2: 71
wandb: 	cnn_layer_size_3: 122
wandb: 	dataset: strib
wandb: 	dropout: 0.10727328652638862
wandb: 	epochs: 25
wandb: 	gru_lay_1: False
wandb: 	gru_lay_2: True
wandb: 	gru_layer_size_1: 219
wandb: 	gru_layer_size_2: 34
wandb: 	gru_layer_size_3: 33
wandb: 	kernelsize: 3
wandb: 	learning_rate: 0.021740576834116496
wandb: 	lookback: 744
wandb: 	momentum: 0.8349192446685972
wandb: 	num_cnn_layers: 2
wandb: 	num_features: 9
wandb: 	num_gru_layers: 2
wandb: 	optimizer: sgd
wandb: 	splitmethod: standard
wandb: 	splitrate: 0.8
wandb: WARNING Ignored wandb.init() arg project when running a sweep


wandb: Ctrl + C detected. Stopping sweep.


In [16]:
wandb.finish()